In [2]:
# Задание 1
# Используем файл keywords.csv.
 
# Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
 
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
 
# geo_data = {
 
#     'Центр': ['москва', 'тула', 'ярославль'],
 
#     'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
#     'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
# }
 
# Результат классификации запишите в отдельный столбец region.
 
import pandas as pd
geo_data = {
 'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] 
}
df = pd.read_csv("keywords.csv")

for key, mass in geo_data.items():
    for city in mass:
        results = df[df.keyword.str.startswith(city)]
        
        for row in results.iterrows():
            idrow = row[0]
            df.at[idrow, 'region'] = key

print(df[df.region == 'Центр'])
print(df[df.region == 'Северо-Запад'])
print(df[df.region == 'Дальний Восток'])


                                                 keyword   shows region
1259                                              москва  153050  Центр
5983                                   москва третий рим   43182  Центр
6075                                           ярославль   41720  Центр
6999                                           москва 24   37459  Центр
7103                                         москва сити   37022  Центр
7339                              москва слезам не верит   35857  Центр
8132                                       москва погода  151701  Центр
11785                                               тула   23898  Центр
15370                                         москвариум   19220  Центр
16175  москва слезам не верит смотреть в хорошем каче...   18407  Центр
18843                                          москва фм   16400  Центр
39920                                       москва карта   15769  Центр
40095                                       москва время   39941

In [3]:
# Задание 2
# Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
# - оценка 2 и меньше - низкий рейтинг
# - оценка 4 и меньше - средний рейтинг
# - оценка 4.5 и 5 - высокий рейтинг
 
# Результат классификации запишите в столбец class
import pandas as pd
df_movies = pd.read_csv("csvs/movies.csv")
df_ratings = pd.read_csv("csvs/ratings.csv")

df_merge = df_ratings.join(df_movies.set_index('movieId'), on='movieId')

rating = df_merge.groupby('movieId')['rating'].mean()

df_merged = df_movies.join(rating, on='movieId')

df_merged.loc[df_merged.rating <= 2, 'rating_level'] = 'low'
df_merged.loc[(df_merged.rating <= 4) & (df_merged.rating > 2), 'rating_level'] = 'middle'
df_merged.loc[df_merged.rating >= 4.5, 'rating_level'] = 'hight'
df_merged



FileNotFoundError: File b'csvs/movies.csv' does not exist

In [4]:
# Задание 3
# Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
# Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
# Ответ дайте в днях.

import pandas as pd
df_ratings = pd.read_csv("csvs/ratings.csv")

df_ratings['count_votes'] = 1

df_grouped = df_ratings.groupby('userId').sum()
users_ids = [u[0] for u in df_grouped[df_grouped.count_votes >= 100].iterrows()] # kinomans

mins = df_ratings[df_ratings.userId.isin(users_ids)].groupby('userId').min(axis='timestamp')
maxs = df_ratings[df_ratings.userId.isin(users_ids)].groupby('userId').max(axis='timestamp')

df_min_max = mins.merge(maxs, on='userId', suffixes=['_min', '_max'])
df_min_max['diff_days'] = (df_min_max.timestamp_max - df_min_max.timestamp_min)/60/60/24

df_min_max.loc['avg'] = df_min_max.mean() # calculate avg
df_min_max

FileNotFoundError: File b'csvs/ratings.csv' does not exist

In [5]:
# Задание 4
# Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". 
# Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. 
# Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, 
# а пройдемся по следующему алгоритму:
 
# 1. В переменную years запишите список из всех годов с 1950 по 2010.
# 2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
#  Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
# - для каждой строки пройдите по всем годам списка years
# - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
# - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
# 3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
# 4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат 
# по убыванию рейтинга

import pandas as pd
df_movies = pd.read_csv("csvs/movies.csv")
df_ratings = pd.read_csv("csvs/ratings.csv")
years = [y for y in range(1950, 2011)]

def production_year(title):    
    for year in years:
        if "%d" % year in title[1]:
            return year
        
    return 1900

df_merge = df_ratings.join(df_movies.set_index('movieId'), on='movieId')
rating = df_merge.groupby('movieId')['rating'].mean()
df_merged = df_movies.join(rating, on='movieId')

for movie in df_merged.iterrows():
    id_movie = movie[0]
    title = movie[1]
    df_merged.at[id_movie, 'year'] = production_year(title)

df_merged.groupby('year')['rating'].mean().sort_values(ascending=False)

FileNotFoundError: File b'csvs/movies.csv' does not exist